# Homework 04. Chanwoo Kim (student ID:20185288)

In [ ]:
#!/usr/bin/env python3

seq_info = dict()
f_csv = open('2019nCoV_genomes.2020_04_14.csv', 'r')
f_csv.readline()
for line in f_csv:
    tokens = line.replace(' ', '_').strip().split(',')
    seq_id = tokens[0]

    collect_date = tokens[-1]
    if len(collect_date) == 7:
        collect_date += '-00'
    geo_location = tokens[4]
    geo_location = geo_location.replace('North_Carolina', 'NC')
    geo_location = geo_location.replace('Illinois', 'IL')
    geo_location = geo_location.replace('San_Francisco', 'CA-SF')
    geo_location = geo_location.replace('"', '')
    isolation_source = tokens[6]
    #print(geo_location)
    if geo_location not in seq_info:
        seq_info[geo_location] = dict()

    seq_info[geo_location][seq_id] = collect_date
f_csv.close()
#Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date
#NC_045512,2020-01-13T00:00:00Z,Severe acute respiratory syndrome-related coronavirus,29903,China,Homo sapiens,,2019-12

#==> 2019nCoV_genomes.2020_04_14.fa <==
#>NC_045512 |Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1| complete genome
#ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTAGATCT

seq_list = dict()
f_fa = open('../data/2019nCoV_genomes.2020_04_14.fa', 'r')
for line in f_fa:
    if line.startswith('>'):
        tmp_h = line.strip().lstrip('>').split()[0]
        seq_list[tmp_h] = []
    else:
        seq_list[tmp_h].append(line.strip())
f_fa.close()

for tmp_loc in sorted(seq_info.keys()):
    if tmp_loc == '':
        continue
    if len(seq_info[tmp_loc]) < 3:
        continue

    for tmp_id, tmp_date in seq_info[tmp_loc].items():
        tmp_seq = ''.join(seq_list[tmp_id])
        print(">%s|%s|%s\n%s" % (tmp_id, tmp_loc.replace(':', '_'), tmp_date, tmp_seq))

In [1]:
#report code

# The modified function to read sequences from a FASTA file for ease of use.
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

#To make a fasta file that has only genomes from US.
covid_genome= "2019nCoV_genomes.2020_04_14.clean.fa"
genome1=read_fasta(covid_genome)
USAgenome = open("2019CoV_USAgenomes.fa","w")
for gid, gseq in genome1.items():
    if "USA" in gid:
        USAgenome.write(gid)
        gseq=gseq+"\n"
        USAgenome.write(gseq)
USAgenome.close()              

f = open("2019nCoV_genomes.2020_04_14.csv", "r")


In [2]:
ref_genome= "GCF_009858895.2_ASM985889v3_genomic.fasta"
GCF_009858895_genome=read_fasta(ref_genome)
f = open("reference.fasta", "w")
for x,y in GCF_009858895_genome.items():
    f.write(x)
f.close()
    

In [1]:
#This file filters the fasta input file to filter the genome sequences that have the geological location within USA
# for multiple sequence alignment via nextstrain
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv
covid_genome= "2019CoV_USAgenomes.fa"
genome=read_fasta(covid_genome)
USAgenome = open("2019CoV_USAgenomes2.fa","w")
for gid, gseq in genome.items():
  
    gid2=gid.split("|")[0]+'\n'
    USAgenome.write(gid2)
    gseq=gseq+"\n"
    USAgenome.write(gseq)
USAgenome.close()   

In [4]:
#Here, for the input file I used the sequence file of the CoV-2 genomes where all the genomes are aligned with mafft program via nextstrain.
gf=open("GCF_009858895.2_ASM985889v3_genomic.gff","r")
glines=gf.readlines()
genename=set()
for r in glines[7:-1]:
    if r.split()[2]=="gene":
        genename.add((r.split()[8].split(';')[2].split('=')[1]))

alignedgenome="aligned.fasta"
genome=read_fasta(alignedgenome)
#From gene information available in gff file, I extracted the sequence information available for each genome, and wrote the fasta file 
# for each aligned gene.
for n in genename:
    genealign=open("%saligned.fasta" % (n),"w")
    for r in glines[9:-2]:
        if n==r.split()[8].split(';')[2].split('=')[1]:
            for gid, gseq in genome.items():
                geneid=gid.rstrip("\n")+n+'\n'
                genealign.write(geneid)
                geneseq=gseq[int(r.split()[3])-1:int(r.split()[4])]+'\n'
                genealign.write(geneseq)
    genealign.close()


In [20]:
#Sequence diversity plot code

alignedgenome="ORF10aligned.fasta" #file to align
genome=read_fasta(alignedgenome)
seqsize=len(genome['>MT258380\n']) 
window_length=10    #size of the sliding window
step=2                #the step size for each itegration of the sliding window shift.       
base=str()
total_score=0      #total diversity to calculate
def my_range(start, end, Step):
    while start <= end:
        yield start
        start += Step
f=open("variabliltytableORF10window10.txt","w")                       #output file
f.write("Nucleotide_position"+'\t'+'Variability_score'+'\n')
  #Multiple loops for the calculation of the diversity score at each sliding window
for x in my_range(1, seqsize-window_length,step):             
    score=0
    f.write("%s" % x+'\t')
    for y in  range(0,window_length):     #At each window shift, the base counts are reset to 0.
        A=0
        C=0
        G=0
        T=0
        N=0
        del base
        z=x+y
#process of how the consensus base is determined by counting each base occurrence
        for gid,gseq in genome.items():         
            if gseq[z]=="A":
                A+=1
            if gseq[z]=="C":
                C+=1
            if gseq[z]=="G":
                G+=1
            if gseq[z]=="T":
                T+=1
            if gseq[z]=="N":
                N+=1
            if gseq[z]=="S":
                G+=1
                C+=1
            if gseq[z]=="W":
                A+=1
                T+=1
            if gseq[z]=="M":
                A+=1
                C+=1
            if gseq[z]=="R":
                A+=1
                G+=1            
            if gseq[z]=="Y":
                T+=1
                C+=1
            if gseq[z]=="K":
                G+=1
                T+=1
            if gseq[z]=="B":
                A-=1
            if gseq[z]=="D":
                C-=1
            if gseq[z]=="H":
                G-=1
            if gseq[z]=="V":
                T-=1
        if max(A,C,G,T,N)==A:
            base="A"
        if max(A,C,G,T,N)==C:
            base="C"
        if max(A,C,G,T,N)==G:
            base="G"
        if max(A,C,G,T,N)==T:
            base="T"
        if max(A,C,G,T,N)==N:
            base="N"
        
#Code to get the score for each sliding window
        for gid,gseq in genome.items():
            if base!=gseq[z]:
                score+=1
            if base=="N" and base!=gseq[z]:
                score-=1
            if base=="A" and gseq[z]==("W"or"M"or "R"):
                score-=0.5
            if base=="C" and gseq[z]==("S" or "M" or "Y"):
                score-=0.5
            if base=="G" and gseq[z]==("S" or"K"or"R"):
                score-=0.5
            if base=="T" and gseq[z]==("Y" or"K"or "W"):
                score-=0.5
            if base=="A" and gseq[z]==("D" or "V" or "H"):
                score-=0.33
            if base=="C" and gseq[z]==("B"or "V"or"H"):
                score-=0.33
            if base=="G" and gseq[z]==("D" or "B" or "V"):
                score-=0.33
            if base=="T" and gseq[z]==("D" or "H" or "V"):
                score-=0.33 
                
    f.write(str(score)+'\n')    
    scorelist.append(score)
    total_score+=score
    print(score)
#This is the output of the code which produces the normalized diversity score for the analyzed protein.
print(total_score/(seqsize-window_length)*step/window_length, "is the normalized diversity score for the protein/genome")

f.close()


1
1
1
1
1
0
1
1
1
1
1
0
0
0
0.5
0.5
0.5
0.5
0.5
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
54
0.04205607476635514 is the diversity score for the protein/genome


In [21]:
#Just to find the set containing the consensus base.
import math
alignedgenome="aligned.fasta"
genome=read_fasta(alignedgenome)
seqsize=len(genome['>MT258380\n'])
window_length=20
step=5
scorelist=list()
baseset=set()

def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

def my_range(start, end, Step):
    while start <= end:
        yield start
        start += Step
        
for x in my_range(1, seqsize-window_length,step):
    score=0
    for y in  range(0,window_length):
        baselist=[]
        z=x+y
        if z+y==1:
            print("analysis start")        
        if z+y==5001:
            print("5001 iterations")
        if z+y==10001:
            print("10001 iterations")
        if z+y==20001:
            print("20001 iterations")            
        for gid,gseq in genome.items():
            baselist.append(gseq[z])
        baseset.add(most_frequent(baselist))
#Output to show the possible consensus bases
print(baseset)

analysis start
5001 iterations
5001 iterations
5001 iterations
5001 iterations
10001 iterations
10001 iterations
10001 iterations
10001 iterations
20001 iterations
20001 iterations
20001 iterations
20001 iterations
{'N', 'T', 'G', 'A', 'C'}
